In [13]:
# Instalar pacotes necessários no Google Colab
!pip install selenium webdriver-manager

# Importar as bibliotecas necessárias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd

# Configurar o driver do Selenium para rodar no Google Colab
options = webdriver.ChromeOptions()
#options.add_argument('--headless')  # Rodar o Chrome em modo headless (sem interface gráfica)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Inicializar o driver do Selenium
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Acessar a URL do formulário
url = "https://www.legislacao.pr.gov.br/legislacao/pesquisarAto.do?action=listar&opt=t&site=1#resultado"  # Substitua pela URL do formulário
driver.get(url)

# Selecionar a opção "Texto Completo"
radio_texto_completo = driver.find_element(By.XPATH, "//input[@value='2' and @name='optTexto']")
radio_texto_completo.click()

# Preencher o campo de termos da pesquisa
input_texto = driver.find_element(By.ID, "texto")
input_texto.send_keys("Código de Ética")

# Preencher o ano inicial e final
ano_inicial = driver.find_element(By.ID, "anoInicialAto")
ano_inicial.send_keys("2010")

ano_final = driver.find_element(By.ID, "anoFinalAto")
ano_final.send_keys("2024")


# Clicar no botão "Marcar Todos"
botao_marcar_todos = driver.find_element(By.ID, "btMarcaDesmarca")
botao_marcar_todos.click()


# Marcar a opção "Lei"
#checkbox_lei = driver.find_element(By.XPATH, "//input[@value='1' and @name='tiposAto']")
#checkbox_lei.click()

# Submeter o formulário clicando no botão "Pesquisar"
botao_pesquisar = driver.find_element(By.ID, "btPesquisar1")
botao_pesquisar.click()

# Aguardar o carregamento da página de resultados
driver.implicitly_wait(60)

# Mostrar o HTML de resposta
html_resposta = driver.page_source
print(html_resposta)


# Salvar o HTML em um arquivo
with open("resposta_pesquisa.html", "w", encoding="utf-8") as file:
    file.write(html_resposta)




# HTML de exemplo
html_content = html_resposta

# Parseando o HTML com BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Listas para armazenar os dados
epigrafe = []
descricao = []
data_publicacao = []

# Loop para extrair os dados
for row in soup.find_all('tr'):
    epigrafe_ato = row.find('div', id=lambda x: x and x.endswith('_epigrafeAto'))
    descricao_item = row.find('div', id=lambda x: x and x.endswith('_descricaoItemAto'))
    data_publicacao_str = row.find('div', id=lambda x: x and x.endswith('_dataPublicacaoStr'))
    
    if epigrafe_ato and descricao_item and data_publicacao_str:
        epigrafe.append(epigrafe_ato.text.strip())
        descricao.append(descricao_item.text.strip())
        data_publicacao.append(data_publicacao_str.text.strip())

# Criando o dataframe
df = pd.DataFrame({
    'Epigrafe': epigrafe,
    'Descricao': descricao,
    'Data_Publicacao': data_publicacao
})

# Salvando em arquivo parquet
df.to_parquet('dados_estruturados_codigo_etica.parquet')

print("Dados extraídos e salvos com sucesso!")



# Fechar o driver do navegador
#driver.quit()


DEPRECATION: Loading egg at d:\anaconda3\lib\site-packages\epajorm-1.0.7-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


<html><head>	
	<title></title>
	<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
	<meta http-equiv="cache-control" content="max-age=0">
	<meta http-equiv="cache-control" content="no-cache">
	<meta http-equiv="expires" content="0">
	<meta http-equiv="expires" content="Tue, 01 Jan 1980 1:00:00 GMT">
	<meta http-equiv="pragma" content="no-cache">
	<link href="css/publico/screen.css" rel="stylesheet" type="text/css">	
	<link href="css/publico/default.css" rel="stylesheet" type="text/css">
	<link href="css/print.css" rel="stylesheet" type="text/css" media="print">
	
	<script language="JavaScript" type="text/JavaScript" src="js/generic/data.js"></script>
	<script language="JavaScript" type="text/JavaScript" src="js/generic/hora.js"></script>
	<script language="JavaScript" type="text/JavaScript" src="js/generic/util.js"></script>
	<script language="JavaScript" type="text/JavaScript" src="js/generic/fone.js"></script>
	<script language="JavaScript" type="text/JavaScript" src

In [14]:
df

,Epigrafe,Descricao,Data_Publicacao
0,Portaria CELEPAR 170 - 09 de Setembro de 2024,Recomposição dos membros doComitê Permanente d...,09/09/2024
1,Portaria FUNEAS 372 - 02 de Setembro de 2024,Designa Comissão Provisória de Revisão e Atual...,03/09/2024
2,Portaria APPA 215 - 16 de Julho de 2024,INSTITUIR COMITÊ DE CONTINGÊNCIA PARA EVENTOS ...,31/07/2024
3,Decreto 5512 - 16 de Abril de 2024,Dispõe sobre o relacionamento da administração...,16/04/2024
4,Lei 21.926 - 11 de Abril de 2024,Consolida a legislação paranaense relativa aos...,11/04/2024
...,...,...,...
148,Resolução SESA 248 - 22 de Maio de 2012,Instituir o Incentivo Financeiro de custeio do...,24/05/2012
149,Lei Complementar 136 - 19 de Maio de 2011,Estabelece a Lei Orgânica da Defensoria Públic...,19/05/2011
150,Decreto 8891 - 29 de Novembro de 2010,"Introduz alterações no Regulamento do\nICMS, a...",29/11/2010
151,Lei Complementar 131 - 29 de Setembro de 2010,Dispõe sobre a\nreestruturação da carreira do ...,29/09/2010


In [7]:
import random
import time

def realizar_pesquisa_legislacao(assunto, pegar_detalhes=True):
    # Instalar pacotes necessários no Google Colab
    !pip install selenium webdriver-manager

    # Importar as bibliotecas necessárias
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.common.keys import Keys
    from bs4 import BeautifulSoup
    import pandas as pd

    # Função para tempos de espera aleatórios
    def esperar_aleatorio(min_segundos=1, max_segundos=5):
        time.sleep(random.uniform(min_segundos, max_segundos))

    # Configurar o driver do Selenium para rodar no Google Colab
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    # Inicializar o driver do Selenium
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Acessar a URL do formulário
    url = "https://www.legislacao.pr.gov.br/legislacao/pesquisarAto.do?action=listar&opt=t&site=1#resultado"
    driver.get(url)
    esperar_aleatorio()

    # Selecionar a opção "Texto Completo"
    radio_texto_completo = driver.find_element(By.XPATH, "//input[@value='2' and @name='optTexto']")
    radio_texto_completo.click()
    esperar_aleatorio()

    # Preencher o campo de termos da pesquisa com o parâmetro assunto, simulando digitação
    input_texto = driver.find_element(By.ID, "texto")
    for letra in assunto:
        input_texto.send_keys(letra)
        esperar_aleatorio(0.1, 0.3)  # Pausa breve entre cada letra

    # Preencher o ano inicial e final com pausas aleatórias
    ano_inicial = driver.find_element(By.ID, "anoInicialAto")
    ano_inicial.send_keys("2010")
    esperar_aleatorio()

    ano_final = driver.find_element(By.ID, "anoFinalAto")
    ano_final.send_keys("2024")
    esperar_aleatorio()

    # Clicar no botão "Marcar Todos"
    botao_marcar_todos = driver.find_element(By.ID, "btMarcaDesmarca")
    botao_marcar_todos.click()
    esperar_aleatorio()

    # Submeter o formulário clicando no botão "Pesquisar"
    botao_pesquisar = driver.find_element(By.ID, "btPesquisar1")
    botao_pesquisar.click()
    esperar_aleatorio(3, 6)  # Esperar mais tempo para a pesquisa carregar

    # Mostrar o HTML de resposta
    html_resposta = driver.page_source

    # Parseando o HTML com BeautifulSoup
    soup = BeautifulSoup(html_resposta, 'html.parser')

    # Listas para armazenar os dados
    epigrafe = []
    descricao = []
    data_publicacao = []
    detalhes = []

    # Armazenar o identificador da janela principal
    janela_principal = driver.current_window_handle

    # Loop para extrair os dados da página principal
    for row in soup.find_all('tr'):
        epigrafe_ato = row.find('div', id=lambda x: x and x.endswith('_epigrafeAto'))
        descricao_item = row.find('div', id=lambda x: x and x.endswith('_descricaoItemAto'))
        data_publicacao_str = row.find('div', id=lambda x: x and x.endswith('_dataPublicacaoStr'))
        botao_exibir = row.find('a', href=lambda x: x and x.startswith("javascript:exibir"))

        if epigrafe_ato and descricao_item and data_publicacao_str and botao_exibir:
            epigrafe.append(epigrafe_ato.text.strip())
            descricao.append(descricao_item.text.strip())
            data_publicacao.append(data_publicacao_str.text.strip())

            if pegar_detalhes:
                # Extrair o ID do item a ser exibido
                id_exibir = botao_exibir['href'].split("'")[1]
                
                # Executar o script para abrir o conteúdo
                driver.execute_script(f"exibir('{id_exibir}');")
                esperar_aleatorio(3, 7)  # Aguardar o carregamento do popup

                detalhe_popup = driver.page_source
                detalhes_soup = BeautifulSoup(detalhe_popup, 'html.parser')

                # Extrair todo o texto da página sem HTML
                detalhe = detalhes_soup.get_text(separator=' ', strip=True)

                # Adicionar os detalhes ao array
                detalhes.append(detalhe if detalhe else "Detalhes não disponíveis")

                driver.execute_script("voltar();")
                esperar_aleatorio(2, 5)
            else:
                detalhes.append("Detalhes não coletados")

    # Criando o dataframe
    df = pd.DataFrame({
        'Epigrafe': epigrafe,
        'Descricao': descricao,
        'Data_Publicacao': data_publicacao,
        'Detalhes': detalhes
    })

    # Salvando em arquivo parquet
    df.to_parquet(f'dados_estruturados_{assunto}.parquet')

    print(f"Dados extraídos e salvos com sucesso para o assunto {assunto}!")

    # Fechar o driver do navegador
    driver.quit()

    return df

# Exemplo de uso
df = realizar_pesquisa_legislacao("Planejamento Estratégico", pegar_detalhes=False)


DEPRECATION: Loading egg at d:\anaconda3\lib\site-packages\epajorm-1.0.7-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Dados extraídos e salvos com sucesso para o assunto Planejamento Estratégico!


In [9]:
df = realizar_pesquisa_legislacao("Governança", pegar_detalhes=False)

DEPRECATION: Loading egg at d:\anaconda3\lib\site-packages\epajorm-1.0.7-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
